# Завдання 1

## Get data

In [ ]:
import gzip
import polars as pl
filename = 'NetworkBackfillImpressions.gz'

In [5]:
with gzip.open("NetworkBackfillImpressions.gz", "rt") as f:
    for i in range(2):
        print(f.readline())
        

CreativeSizeDelivered,ImpressionId,Advertiser,Buyer,SellerReservePrice,AdxAccountId,Time,UserId,NativeStyle,AdvertiserId,OrderId,LineItemId,CreativeId,CreativeVersion,CreativeSize,CustomTargeting,AdUnitId,Country,Browser,OS,Domain,BrowserId,MobileDevice,RequestedAdUnitSizes,AudienceSegmentIds,City,BandWidth,GfpContentId,OSVersion,MobileCapability,MobileCarrier,MobileAppId,YieldGroupCompanyId,YieldGroupNames,EstimatedBackfillRevenue,RefererURL,EventKeyPart,EventTimeUsec2,ActiveViewEligibleImpression,IsInterstitial,DeviceCategory,TimeUsec2,KeyPart,TargetedCustomCriteria,VideoFallbackPosition,PodPosition,VideoPosition,IsCompanion,PublisherProvidedID,Product,BandwidthGroupId

300x250,1-0,,PubMatic (EB),1.19,242950182,2020-08-04-15:03:01,cYu18UGeb_nKoij1jTYeHQ,,0,0,0,0,0,,hb_adid=1797626ee14ffc85;hb_bidder=ix;hb_format=banner;hb_pb=1.24;mediumid=192282_1000_com.aol.mobile.aolapp;platform=smartphone;postid=192282;sessionid=192282_1000;sessionid2=192282_1000_com.aol.mobile.aolapp;site=eng.amo

In [6]:
network_impressions = pl.read_csv(filename)

## EDA

Дослідження даних потрібне для кращого розуміння датасету та для того, аби обрати найкращий підхід до виконання цього завдання. 
Можна виділити 2 задачі:
1. Потрібно визначити, де брати session_id (за стовбцем dfpsessionid)
2. Перевірити, чи немає порожніх значень у "EstimatedBackfillRevenue", "DeviceCategory", "Product" (для групування по завданню)

Так як окремого стовбця-сутності для позначення сесій не знайдено, то це значення може бути у параметрі

### Визначення потрібного параметра для session_id

Нижче реалізація пошуку параметра у всьому датасеті. Визначили, що наш параметр знаходиться у CustomTargeting

In [7]:
candidate_columns = []

#dfpsessionid, sessionid, sessionid2
for col, dtype in zip(network_impressions.columns, network_impressions.dtypes):
    if dtype == pl.Utf8:
        contains_dfpsession = (
            network_impressions
            .select(pl.col(col).str.contains("dfpsessionid="))
            .to_series()
            .any()
        )

        if contains_dfpsession:
            candidate_columns.append(col)

candidate_columns

['CustomTargeting']

Нижче бачимо, що також у цьому рядку крім dfpsessionid є ще sessionid та sessionid2. Краще проаналізувати усі 3 параметри, щоб обрати стратегію витягнення ідентифікатора. Для цього я витягнув кожен параметр та підрахував рядки за комбінаціями

In [8]:
network_impressions.select("CustomTargeting").head(1).to_series()[0]

'hb_adid=1797626ee14ffc85;hb_bidder=ix;hb_format=banner;hb_pb=1.24;mediumid=192282_1000_com.aol.mobile.aolapp;platform=smartphone;postid=192282;sessionid=192282_1000;sessionid2=192282_1000_com.aol.mobile.aolapp;site=eng.amomedia.com;traffic_source=yahoo;utm=yahoo'

In [9]:
network_impressions.select([
    pl.col("CustomTargeting")
      .str.contains(r"sessionid=")
      .sum()
      .alias("has_sessionid"),

    pl.col("CustomTargeting")
      .str.contains(r"dfpsessionid=")
      .sum()
      .alias("has_dfpsessionid"),

    pl.col("CustomTargeting")
      .str.contains(r"sessionid2=")
      .sum()
      .alias("has_sessionid2"),
])

has_sessionid,has_dfpsessionid,has_sessionid2
u32,u32,u32
740343,71752,585870


За результатом бачимо, що у вибірці не завжди ці 3 параметра присутні. session_id присутній у 75% вибірки, dfpsessionid лише у 7%, а sessionid2 у +-60%. У нашому випадку це не ок, адже більшість даних буде просто не валідно для обрахунку у поточному завданні. Треба проаналізувати саме кореляцію, коли ці поля зустрічаються між собою

In [10]:
summary_table = (
    network_impressions
    .select([
        pl.col("CustomTargeting")
          .str.contains(r"(?:^|;)sessionid=")
          .alias("has_sessionid"),

        pl.col("CustomTargeting")
          .str.contains(r"(?:^|;)dfpsessionid=")
          .alias("has_dfpsessionid"),

        pl.col("CustomTargeting")
          .str.contains(r"(?:^|;)sessionid2=")
          .alias("has_sessionid2"),
    ])
    .group_by([
        "has_sessionid",
        "has_dfpsessionid",
        "has_sessionid2",
    ])
    .count()
)

summary_table

C:\Users\trofy\AppData\Local\Temp\ipykernel_524\4052186404.py:21: DeprecationWarning: `GroupBy.count` was renamed; use `GroupBy.len` instead
  .count()


has_sessionid,has_dfpsessionid,has_sessionid2,count
bool,bool,bool,u32
false,true,false,4533
true,false,false,84243
true,true,true,1522
null,null,null,6773
false,false,false,229745
true,true,false,65697
true,false,true,584348


Вище бачимо закономірність, якщо sessionid присутній, то і майже завжди присутні інші ідентифікатори.
У нашому варіанті краще брати sessionid, аніж dfpsessionid - в залежності від специфіки сорсу, але даний варіант забезпечує зберігання 75% вибірки
Але також бачимо ситуацію, що є випадки, коли sessionid відсутній, а dfpsessionid присутній.
Можна зробити наступний висновок щодо вибору нашого параметру сесії:
1. У більшості випадків краще брати параметр sessionid
2. Якщо sessionid порожній - шукаємо непорожнє значення dfpsessionid
3. Якщо dfpsessionid порожній - метчимо як "unknown"

Чому не варто розглядати sessionid2:
У нашому кейсі:
1. sessionid2 не збільшує покриття
2. створює ризик різних семантик сесії - при роботі у проді
3. не дає нових кейсів без sessionid
Його використання на даному етапі - це overengineering і не зрозуміла його бізнес-цінність

### Перевірка значень стовбців на пропуски

In [11]:
network_impressions["EstimatedBackfillRevenue", "DeviceCategory", "Product"].describe()

statistic,EstimatedBackfillRevenue,DeviceCategory,Product
str,f64,str,str
"""count""",976861.0,"""976861""","""976861"""
"""null_count""",0.0,"""0""","""0"""
"""mean""",0.00087,null,null
"""std""",0.001392,null,null
"""min""",0.000005,"""Connected TV""","""Ad Exchange"""
"""25%""",0.000228,null,null
"""50%""",0.000454,null,null
"""75%""",0.000952,null,null
"""max""",0.290525,"""Tablet""","""Exchange Bidding"""


Бачимо, що пропусків немає - можемо працювати

## Realize

In [12]:
network_impressions = network_impressions.with_columns(
    pl.coalesce(
        pl.col("CustomTargeting").str.extract(r"(?:^|;)sessionid=([^;]+)"),
        pl.col("CustomTargeting").str.extract(r"(?:^|;)dfpsessionid=([^;]+)"),
        pl.lit("unknown")
    ).alias("session_id")
)

In [13]:
network_impressions['session_id'].is_null().sum()

0

In [14]:
sessions_summary = (
    network_impressions.group_by(["Product", "DeviceCategory", "session_id"])
    .agg(
        [
            pl.sum("EstimatedBackfillRevenue").alias("total_cost"),
            pl.count("ImpressionId").alias("impression_count")
        ]).sort("total_cost", descending=True)
)
sessions_summary

Product,DeviceCategory,session_id,total_cost,impression_count
str,str,str,f64,u32
"""Ad Exchange""","""Smartphone""","""unknown""",127.689951,165065
"""Ad Exchange""","""Smartphone""","""192645_2002""",32.730354,76026
"""Ad Exchange""","""Smartphone""","""191939_2013""",27.027685,42960
"""Ad Exchange""","""Desktop""","""unknown""",22.488907,23703
"""Exchange Bidding""","""Smartphone""","""unknown""",18.55915,18502
…,…,…,…,…
"""Ad Exchange""","""Smartphone""","""9064fcce-e47d-4a83-83f5-ffb9a8…",0.000014,1
"""Ad Exchange""","""Desktop""","""191794_3056""",0.000014,1
"""Ad Exchange""","""Desktop""","""191767_3066""",0.000014,1


In [15]:
sessions_summary.write_csv("sessions_summary.csv")
sessions_summary.write_parquet("sessions_summary.parquet")